In [ ]:
include(Pkg.dir("Dyn3d")*"/src/Dyn3d.jl")
using Dyn3d

# choose config files from folder Config_files
include("../src/Config_files/2dLink.jl")
# include("../src/Config_files/2dFall.jl")
# include("../src/Config_files/2dSwim.jl")
# include("../src/Config_files/3dHinge.jl")
# include("../src/Config_files/3dPrismatic.jl")
# include("../src/Config_files/3dCylindrical.jl")
# include("../src/Config_files/2dThrow.jl")

In [ ]:
# add bodys
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

# add joints
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

# assemble system to a chain
system = System(ndim, nbody, njoint, gravity, num_params)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

In [ ]:
# # test function UpdatePosition!
# bodys, joints, system = UpdatePosition!(bodys, joints, system)

# # test function UpdateVelocity!
# for i = 1:nbody
#     @assert length(bodys[i].v) == 6
#     @assert length(bodys[i].Xp_to_b) == 36
#     @assert length(joints[i].vJ) == 6
# end
# v = ones(Float64,24)
# bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)

In [ ]:
# init system
bodys, joints, system, soln = InitSystem!(bodys, joints, system, scheme)

# init soln structure
solns = (Soln)[]
push!(solns, soln)

# init VertsHistory struct
vs = []
push!(vs, VertsHistory(system.nbody, bodys))

In [ ]:
# advance in time
idx = 0
@time begin
while soln.t < tf
    # advance one step
    soln, bodys, joints, system = HERK!(soln, bodys, joints, system, scheme, tol)
        
    # record soln and verts_i info
    push!(solns, soln)
    push!(vs, VertsHistory(system.nbody, bodys))
        
    # print progress
    idx += 1
    if mod(idx,500) == 1 
        @printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
#         println("center of mass at", MassCenter(bodys, system))
    end
end
@printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt)
end

In [ ]:
using MAT
using Interpolations

# create regular time grid and acquire solutions on it
qJ_regs = Float64[]
t_reg = linspace(0,solns[end].t,length(solns))
for i = 1:system.ndof
    t_temp = ([solns[k].t for k = 1:length(solns)],)
    qJ_temp = [solns[k].qJ[i] for k = 1:length(solns)]   
    qJ_reg = interpolate(t_temp, qJ_temp, Gridded(Linear()))[t_reg]
    append!(qJ_regs, qJ_reg)
end
qJ_regs = reshape(qJ_regs,(length(solns), system.ndof))

# get verts info based on this regular grid solution
vs_reg = []
bodys_reg = deepcopy(bodys)
joints_reg = deepcopy(joints)
system_reg = deepcopy(system)
for i = 1:length(solns)
    bodys_reg, joints_reg, system_reg = UpdatePosition!(bodys_reg, joints_reg, system_reg, solns[i].qJ)
    push!(vs_reg, VertsHistory(system.nbody, bodys_reg))
end

# write to .mat file for animation
matwrite("../matlab_plot/verts_i.mat", Dict(
    "ndim" => system.ndim,
    "nbody" => system.nbody,
    "nverts" => bodys[1].nverts,
    "t" => collect(t_reg),
    "verts" => vs_reg
))

In [ ]:
# # results printed
# solns[end].qJ
# solns[end].v
# solns[end].v̇
# solns[end].λ
# solns[end].t

# # code analysis
# using BenchmarkTools, Compat
# @benchmark HERK!(soln, bodys, joints, system, scheme, tol)
# @profile HERK!(soln, bodys, joints, system, scheme, tol)
# Profile.print()

# using Coverage
# m = analyze_malloc("../src")
# m[end-20:end]